In [18]:
import cv2 as cv
import numpy as np
import argparse
import imageio
import matplotlib.pyplot as plt
%matplotlib widget

W = 30          # window size is WxW
C_Thr = 0.8   # threshold for coherency
time_pixel_size = 0.1 #s.pixel
space_pixel_size = 0.267 #um.pixel
intensity_volume_factor = 1 #um**3.intensity-unit

def calcGST(inputIMG, w):
    img = inputIMG.astype(np.float32)
    imgDiffX = cv.Sobel(img, cv.CV_32F, 1, 0, 3)
    imgDiffY = cv.Sobel(img, cv.CV_32F, 0, 1, 3)
    imgDiffXY = cv.multiply(imgDiffX, imgDiffY)
    
    imgDiffXX = cv.multiply(imgDiffX, imgDiffX)
    imgDiffYY = cv.multiply(imgDiffY, imgDiffY)
    J11 = cv.boxFilter(imgDiffXX, cv.CV_32F, (w,w))
    J22 = cv.boxFilter(imgDiffYY, cv.CV_32F, (w,w))
    J12 = cv.boxFilter(imgDiffXY, cv.CV_32F, (w,w))
    tmp1 = J11 + J22
    tmp2 = J11 - J22
    tmp2 = cv.multiply(tmp2, tmp2)
    tmp3 = cv.multiply(J12, J12)
    tmp4 = np.sqrt(tmp2 + 4.0 * tmp3)
    lambda1 = 0.5*(tmp1 + tmp4)    # biggest eigenvalue
    lambda2 = 0.5*(tmp1 - tmp4)    # smallest eigenvalue
    imgCoherencyOut = cv.divide(lambda1 - lambda2, lambda1 + lambda2)
    imgOrientationOut = cv.phase(J22 - J11, 2.0 * J12, angleInDegrees = True)
    imgOrientationOut = 0.5 * imgOrientationOut
    return imgCoherencyOut, imgOrientationOut
path = r'C:\Users\coren\AMOLF-SHIMIZU Dropbox\Projects\Jaap_Data\ExNo_110_Plate262_C2_NileRed_60x_T_Kymographs_PNG\220330_ExNo_110_60x_C2_Nile red_PLate262-T_15-50_0101_2_kymo_2.png'
imgIn = imageio.imread(path)
imgcrop = imgIn
imgCoherency, imgOrientation = calcGST(imgcrop, W)

In [19]:
fig, ax = plt.subplots()
ax.imshow(imgcrop,cmap="gray")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
fig, ax = plt.subplots()
ax.imshow(imgCoherency,cmap="jet")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
nans = np.empty(imgOrientation.shape)
nans.fill(np.nan)
real_movement = np.where(imgCoherency>C_Thr,imgOrientation,nans)

speeds = np.tan((np.nanmean(real_movement,axis=1)-90)/180*np.pi)*space_pixel_size/time_pixel_size #um.s-1
nans = np.empty(speeds.shape)
nans.fill(np.nan)
speeds = np.where(speeds<20,speeds,nans)
nans = np.empty(speeds.shape)
nans.fill(np.nan)
speeds = np.where(speeds>-20,speeds,nans)

C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: Mean of empty slice
  """


In [22]:
import numpy as np

def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.

    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        >>> # linear interpolation of NaNs
        >>> nans, x= nan_helper(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """

    return np.isnan(y), lambda z: z.nonzero()[0]

y= speeds
nans, x= nan_helper(y)
y[nans]= np.interp(x(nans), x(~nans), y[~nans])
speeds=y

In [32]:
fig, ax = plt.subplots()
ax.imshow(imgcrop,cmap="gray")
ax2 = ax.twiny()
# ax2.plot(np.tan((np.mean(imgOrientation,axis=1)-90)/180*np.pi),range(len(np.mean(imgOrientation,axis=1))),color = "red")
ax2.plot(speeds,range(len(speeds)),color = "red")

ax2.tick_params(axis='x', colors='red')
ax2.set_xlabel('speed')

ax.imshow(imgOrientation*(imgCoherency>C_Thr),alpha=0.5,cmap="jet")
ax2.set_xlim((-20,20))

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
infinitesimal_volume = np.mean(imgcrop,axis=1)*intensity_volume_factor/space_pixel_size #um**3.um-1

In [25]:
fig, ax = plt.subplots()
ax.imshow(imgcrop,cmap="gray")
ax2 = ax.twiny()
ax2.tick_params(axis='x', colors='red')
ax2.set_xlabel('volume')
# ax2.plot(np.tan((np.mean(imgOrientation,axis=1)-90)/180*np.pi),range(len(np.mean(imgOrientation,axis=1))),color = "red")
ax2.plot(infinitesimal_volume,range(len(infinitesimal_volume)),color = "red")
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
flux = infinitesimal_volume*speeds #um**3.s-1
fig, ax = plt.subplots()
ax.imshow(imgcrop,cmap="gray")
ax2 = ax.twiny()
ax2.tick_params(axis='x', colors='red')
ax2.set_xlabel('flux')
# ax2.plot(np.tan((np.mean(imgOrientation,axis=1)-90)/180*np.pi),range(len(np.mean(imgOrientation,axis=1))),color = "red")
ax2.plot(flux,range(len(infinitesimal_volume)),color = "red")
plt.tight_layout()
ax2.set_xlim((-8000,8000))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-8000.0, 8000.0)

In [33]:
print("mean flux (um**3.s-1)", np.nanmean(flux),np.nanstd(flux)/np.sqrt(len(flux)))
print("mean speed (um.s-1)", np.nanmean(speeds),np.nanstd(speeds)/np.sqrt(len(speeds)))


mean flux (um**3.s-1) 176.6443430177544 12.758639820790272
mean speed (um.s-1) 1.1954827339623522 0.04871394094284938


In [31]:
from scipy.fft import fft, fftfreq

# Number of sample points
N = len(speeds)
x = np.linspace(0.0, N*time_pixel_size, N, endpoint=False)

y = (speeds-np.mean(speeds))/np.std(speeds)
yf = fft(y)
T = time_pixel_size
from scipy.signal import blackman

w = blackman(N)

ywf = fft(y*w)

xf = fftfreq(N, T)[:N//2]

import matplotlib.pyplot as plt
fig, ax = plt.subplots()

ax.semilogy(xf[1:N//2], 2.0/N * np.abs(yf[1:N//2]), '-b')

# ax.semilogy(xf[1:N//2], 2.0/N * np.abs(ywf[1:N//2]), '-r')

ax.legend(['FFT', 'FFT w. window'])

ax.grid()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[result.size // 2:]
fig, ax = plt.subplots()
autoc = autocorr(y)
autoc = autoc/autoc.max()
lags = np.linspace(0.0, len(autoc)*time_pixel_size, len(autoc), endpoint=False)
ax.plot(lags,autoc, label = 'savgol filter')
ax.set_xlabel('lag (s)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'lag (s)')

In [32]:
fig, ax = plt.subplots()
autoc = autocorr(y)
autoc = autoc/autoc.max()
lags = np.linspace(0.0, len(autoc)*time_pixel_size, len(autoc), endpoint=False)
ax.plot(lags,autoc, label = 'savgol filter')
ax.set_xlabel('lag (s)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'lag (s)')

In [11]:
N = len(autoc)
x = np.linspace(0.0, N*time_pixel_size, N, endpoint=False)

y = (speeds-np.mean(autoc))/np.std(autoc)
yf = fft(y)
T = time_pixel_size
from scipy.signal import blackman

w = blackman(N)

ywf = fft(y*w)

xf = fftfreq(N, T)[:N//2]

import matplotlib.pyplot as plt
fig, ax = plt.subplots()

ax.semilogy(xf[1:N//2], 2.0/N * np.abs(yf[1:N//2]), '-b')

ax.semilogy(xf[1:N//2], 2.0/N * np.abs(ywf[1:N//2]), '-r')

ax.legend(['FFT', 'FFT w. window'])

ax.grid()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …